Guignard Quentin, Stone Tomas, Hanus Maxime

# Question 1 : Explication de type (1-1) pour la Comparaison x > y

## Formulation Mathématique

### 1. Données et Paramètres
- **x, y** : vecteurs de notes des deux candidats pour chaque cours i
- **w** : vecteur des poids de chaque cours
- **contributions(i)** = w[i] × (x[i] - y[i]) : contribution du cours i à la somme pondérée

### 2. Ensembles
- **pros(x,y)** : ensemble des cours i où x[i] > y[i] (contributions > 0)
- **cons(x,y)** : ensemble des cours i où x[i] < y[i] (contributions < 0)
- **neutral(x,y)** : ensemble des cours i où x[i] = y[i] (contributions = 0)

### 3. Formulation du PL

**Variables de décision:**
- Pour chaque paire (P, C) où P ∈ pros(x,y) et C ∈ cons(x,y) :
  - $x_{P,C} ∈ {0, 1}$ : égal à 1 si la paire (P, C) est sélectionnée comme trade-off

**Contraintes:**
1. Chaque cours $C \in \text{cons}(x,y)$ doit être appairé exactement une fois :
   $$\sum_{P \in \text{pros}(x,y)} x_{P,C} = 1 \quad \forall C \in \text{cons}(x,y)$$

2. Chaque cours $P \in text{pros}(x,y)$ peut être appairé au plus une fois :
   $$\sum_{C \in \text{cons}(x,y)} x_{P,C} \leq 1 \quad \forall P \in \text{pros}(x,y)$$

3. Chaque trade-off sélectionné doit être valide (contribution(P) + contribution(C) > 0) :
   $$(\text{contribution}(P) + \text{contribution}(C)) \cdot x_{P,C} > 0$$
   
   Ou de façon linéaire (si toutes les paires positives) :
   $$\text{contribution}(P) + \text{contribution}(C) > 0 \quad \text{pour chaque paire sélectionnée}$$

**Objectif:**
- Minimiser le nombre de trade-offs : $\min \sum_{P,C} x_{P,C}$
- Ou maximiser pour chercher une explication quelconque : $\max \sum_{P,C} x_{P,C}$

**Certificat de non-existence:**
Si le PL est infaisable (aucune solution), il n'existe pas d'explication (1-1).

In [7]:
import numpy as np
from gurobipy import *

# Données
courses = ['Anatomie', 'Biologie', 'Chirurgie', 'Diagnostic', 'Epidemiologie', 'Forensic', 'Genetique']
x_notes = np.array([85, 81, 71, 69, 75, 81, 88])
y_notes = np.array([81, 81, 75, 63, 67, 88, 95])
z_notes = np.array([74, 89, 74, 81, 68, 84, 79])
t_notes = np.array([74, 71, 84, 91, 77, 76, 73])
u_notes = np.array([72, 75, 66, 85, 88, 66, 93])
v_notes = np.array([71, 73, 63, 92, 76, 79, 93])
w_notes = np.array([79, 69, 78, 76, 67, 84, 79])
w_prime_notes = np.array([57, 76, 81, 76, 82, 86, 77])
weights = np.array([8, 7, 7, 6, 6, 5, 6])

# Contributions pondérées
contributions = weights * (x_notes - y_notes)
pros = [i for i in range(len(courses)) if contributions[i] > 0]
cons = [i for i in range(len(courses)) if contributions[i] < 0]
neutral = [i for i in range(len(courses)) if contributions[i] == 0]

print('pros:  ', [courses[i] for i in pros])
print('cons:  ', [courses[i] for i in cons])
print('neutral:', [courses[i] for i in neutral])

# Trade-offs (1-1) valides: contribution(P)+contribution(C) > 0
valid_tradeoffs = []
for p in pros:
    for c in cons:
        s = contributions[p] + contributions[c]
        if s > 0:
            valid_tradeoffs.append((p, c, s))

print('trade-offs valides:', [(courses[p], courses[c], s) for p, c, s in valid_tradeoffs])

pros:   ['Anatomie', 'Diagnostic', 'Epidemiologie']
cons:   ['Chirurgie', 'Forensic', 'Genetique']
neutral: ['Biologie']
trade-offs valides: [('Anatomie', 'Chirurgie', 4), ('Diagnostic', 'Chirurgie', 8), ('Diagnostic', 'Forensic', 1), ('Epidemiologie', 'Chirurgie', 20), ('Epidemiologie', 'Forensic', 13), ('Epidemiologie', 'Genetique', 6)]


In [8]:
# Résolution PL pour une explication (1-1) minimale
m = Model('explication_1_1')
x_vars = {}
for p, c, _ in valid_tradeoffs:
    x_vars[(p,c)] = m.addVar(vtype=GRB.BINARY, name=f'x_{p}_{c}')
m.update()

# Chaque cons est couvert exactement une fois
for c in cons:
    pairs = [(p, cc) for (p, cc, _) in valid_tradeoffs if cc == c]
    if pairs:
        m.addConstr(quicksum(x_vars[(p, cc)] for (p, cc) in pairs) == 1)

# Chaque pro est utilisé au plus une fois
for p in pros:
    pairs = [(pp, c) for (pp, c, _) in valid_tradeoffs if pp == p]
    if pairs:
        m.addConstr(quicksum(x_vars[(pp, c)] for (pp, c) in pairs) <= 1)

# Objectif: minimiser le nombre de paires
m.setObjective(quicksum(x_vars.values()), GRB.MINIMIZE)
m.params.outputflag = 0
m.optimize()

if m.status == GRB.OPTIMAL:
    print('OPTIMAL, longueur =', int(m.objVal))
    chosen = []
    for (p, c), var in x_vars.items():
        if var.X > 0.5:
            chosen.append((p, c))
            pc = contributions[p]
            cc = contributions[c]
            print(f'({courses[p]}, {courses[c]}) -> {pc} + ({cc}) = {pc+cc}')
else:
    print('Pas de solution (certificat de non-existence).')

OPTIMAL, longueur = 3
(Anatomie, Chirurgie) -> 32 + (-28) = 4
(Diagnostic, Forensic) -> 36 + (-35) = 1
(Epidemiologie, Genetique) -> 48 + (-42) = 6


# Question 2 : Explication de type (1-m) pour la Comparaison x > y

## Formulation Mathématique

### 1. Définition d'un Trade-off (1-m)
Un trade-off (1-m) est une paire (P, {C₁, ..., Cₘ}) où :
- P ∈ pros(x,y) : un cours où x fait mieux que y
- {C₁, ..., Cₘ} ⊂ cons(x,y) : m cours où y fait mieux que x
- La somme des contributions est positive : contribution(P) + Σⱼ contribution(Cⱼ) > 0

### 2. Explication (1-m)
Une explication (1-m) de x > y est un ensemble de trade-offs (1-m) disjoints qui couvrent tous les cons(x,y).

### 3. Formulation du Programme Linéaire

**Variables de décision:**
- Pour chaque P ∈ pros(x,y) et chaque C ∈ cons(x,y) :
  - $x_{P,C} ∈ \{0, 1\}$ : égal à 1 si le cours C est associé au cours P dans un trade-off

**Contraintes:**
1. Chaque cours $C \in \text{cons}(x,y)$ doit être appairé exactement une fois :
   $$\sum_{P \in \text{pros}(x,y)} x_{P,C} = 1 \quad \forall C \in \text{cons}(x,y)$$

2. Pour chaque cours $P \in \text{pros}(x,y)$, si P est utilisé, son trade-off doit être valide :
   $$\text{contribution}(P) + \sum_{C \in \text{cons}(x,y)} \text{contribution}(C) \cdot x_{P,C} > 0$$
   
   Version linéarisée avec $y_P \in \{0,1\}$ (indicateur si P est utilisé) :
   $$\text{contribution}(P) \cdot y_P + \sum_{C \in \text{cons}(x,y)} \text{contribution}(C) \cdot x_{P,C} \geq \epsilon \cdot y_P$$
   
   où $\epsilon > 0$ est une petite constante (e.g., 0.001).

3. Lien entre $x_{P,C}$ et $y_P$ :
   $$x_{P,C} \leq y_P \quad \forall P \in \text{pros}(x,y), C \in \text{cons}(x,y)$$
   $$y_P \leq \sum_{C \in \text{cons}(x,y)} x_{P,C} \quad \forall P \in \text{pros}(x,y)$$

**Objectif:**
- Minimiser le nombre de trade-offs : $\min \sum_{P \in \text{pros}(x,y)} y_P$

**Certificat de non-existence:**
Si le PL est infaisable, il n'existe pas d'explication (1-m).

In [9]:
# Implémentation du PL pour explication (1-m)
def find_explanation_1_m(x_notes, y_notes, weights, courses, candidate_x='X', candidate_y='Y'):
    """
    Trouve une explication (1-m) minimale pour x > y
    Retourne: (status, explanation) où status est 'OPTIMAL' ou 'INFEASIBLE'
    """
    # Contributions pondérées
    contributions = weights * (x_notes - y_notes)
    pros = [i for i in range(len(courses)) if contributions[i] > 0]
    cons = [i for i in range(len(courses)) if contributions[i] < 0]
    
    print(f'\n=== Analyse {candidate_x} > {candidate_y} ===')
    print(f'pros({candidate_x},{candidate_y}):  ', [courses[i] for i in pros])
    print(f'cons({candidate_x},{candidate_y}):  ', [courses[i] for i in cons])
    print(f'Contributions: {contributions}')
    
    if not cons:
        print(f'{candidate_x} domine {candidate_y} sur tous les critères.')
        return 'TRIVIAL', []
    
    if not pros:
        print(f'{candidate_y} domine {candidate_x} sur tous les critères.')
        return 'IMPOSSIBLE', []
    
    # Modèle PL
    m = Model('explication_1_m')
    m.params.outputflag = 0
    
    # Variables: x[p,c] = 1 si C est associé à P
    x_vars = {}
    for p in pros:
        for c in cons:
            x_vars[(p,c)] = m.addVar(vtype=GRB.BINARY, name=f'x_{p}_{c}')
    
    # Variables: y[p] = 1 si P est utilisé dans un trade-off
    y_vars = {}
    for p in pros:
        y_vars[p] = m.addVar(vtype=GRB.BINARY, name=f'y_{p}')
    
    m.update()
    
    epsilon = 0.001
    
    # Contrainte 1: Chaque cons est couvert exactement une fois
    for c in cons:
        m.addConstr(quicksum(x_vars[(p,c)] for p in pros) == 1, name=f'cover_cons_{c}')
    
    # Contrainte 2: Validité des trade-offs
    for p in pros:
        sum_contrib = contributions[p] * y_vars[p]
        for c in cons:
            sum_contrib += contributions[c] * x_vars[(p,c)]
        m.addConstr(sum_contrib >= epsilon * y_vars[p], name=f'valid_tradeoff_{p}')
    
    # Contrainte 3: Lien entre x et y
    for p in pros:
        for c in cons:
            m.addConstr(x_vars[(p,c)] <= y_vars[p], name=f'link1_{p}_{c}')
        m.addConstr(y_vars[p] <= quicksum(x_vars[(p,c)] for c in cons), name=f'link2_{p}')
    
    # Objectif: minimiser le nombre de trade-offs
    m.setObjective(quicksum(y_vars.values()), GRB.MINIMIZE)
    m.optimize()
    
    if m.status == GRB.OPTIMAL:
        print(f'\n✓ Explication (1-m) trouvée, longueur = {int(m.objVal)}')
        explanation = []
        for p in pros:
            if y_vars[p].X > 0.5:
                cons_in_tradeoff = [c for c in cons if x_vars[(p,c)].X > 0.5]
                contrib_p = contributions[p]
                contrib_cons = [contributions[c] for c in cons_in_tradeoff]
                total = contrib_p + sum(contrib_cons)
                explanation.append((p, cons_in_tradeoff))
                print(f'  Trade-off: ({courses[p]}) vs {[courses[c] for c in cons_in_tradeoff]}')
                print(f'    Contributions: {contrib_p:.1f} + {contrib_cons} = {total:.1f}')
        return 'OPTIMAL', explanation
    else:
        print(f'\n✗ Pas d\'explication (1-m) (certificat de non-existence)')
        return 'INFEASIBLE', []

# Test avec Xavier vs Yvonne
result, expl = find_explanation_1_m(x_notes, y_notes, weights, courses, 'Xavier', 'Yvonne')


=== Analyse Xavier > Yvonne ===
pros(Xavier,Yvonne):   ['Anatomie', 'Diagnostic', 'Epidemiologie']
cons(Xavier,Yvonne):   ['Chirurgie', 'Forensic', 'Genetique']
Contributions: [ 32   0 -28  36  48 -35 -42]

✓ Explication (1-m) trouvée, longueur = 3
  Trade-off: (Anatomie) vs ['Chirurgie']
    Contributions: 32.0 + [-28] = 4.0
  Trade-off: (Diagnostic) vs ['Forensic']
    Contributions: 36.0 + [-35] = 1.0
  Trade-off: (Epidemiologie) vs ['Genetique']
    Contributions: 48.0 + [-42] = 6.0


## Démonstration : Absence d'explication (1-m) pour u > v

Nous allons maintenant prouver qu'il n'existe pas d'explication (1-m) pour la comparaison u > v.

In [10]:
# Test de l'explication (1-m) pour u > v
result_uv, expl_uv = find_explanation_1_m(u_notes, v_notes, weights, courses, 'u', 'v')


=== Analyse u > v ===
pros(u,v):   ['Anatomie', 'Biologie', 'Chirurgie', 'Epidemiologie']
cons(u,v):   ['Diagnostic', 'Forensic']
Contributions: [  8  14  21 -42  72 -65   0]

✗ Pas d'explication (1-m) (certificat de non-existence)


## Explication de type (m-1) pour y > z

Un trade-off (m-1) est une paire ({P₁, ..., Pₘ}, C) où :
- {P₁, ..., Pₘ} ⊂ pros(y,z) : m cours où y fait mieux que z
- C ∈ cons(y,z) : un cours où z fait mieux que y
- La somme des contributions est positive : Σⱼ contribution(Pⱼ) + contribution(C) > 0

### Formulation du Programme Linéaire pour (m-1)

**Variables:**
- $x_{P,C} ∈ \{0, 1\}$ : égal à 1 si P est associé au cons C
- $y_C ∈ \{0, 1\}$ : égal à 1 si C est utilisé dans un trade-off

**Contraintes:**
1. Chaque P ∈ pros(y,z) est associé à exactement un C :
   $$\sum_{C \in \text{cons}(y,z)} x_{P,C} = 1 \quad \forall P \in \text{pros}(y,z)$$

2. Validité des trade-offs (pour chaque C utilisé) :
   $$\sum_{P \in \text{pros}(y,z)} \text{contribution}(P) \cdot x_{P,C} + \text{contribution}(C) \cdot y_C \geq \epsilon \cdot y_C$$

3. Lien entre x et y :
   $$x_{P,C} \leq y_C \quad \forall P, C$$
   $$y_C \leq \sum_{P \in \text{pros}(y,z)} x_{P,C} \quad \forall C$$

**Objectif:** $\min \sum_C y_C$

In [ ]:
# Implémentation du PL pour explication (m-1)
def find_explanation_m_1(x_notes, y_notes, weights, courses, candidate_x='X', candidate_y='Y'):
    """
    Trouve une explication (m-1) minimale pour x > y
    En (m-1), chaque cons doit être couvert par exactement un trade-off.
    Retourne: (status, explanation)
    """
    # Contributions pondérées
    contributions = weights * (x_notes - y_notes)
    pros = [i for i in range(len(courses)) if contributions[i] > 0]
    cons = [i for i in range(len(courses)) if contributions[i] < 0]
    
    print(f'\n=== Analyse {candidate_x} > {candidate_y} (type m-1) ===')
    print(f'pros({candidate_x},{candidate_y}):  ', [courses[i] for i in pros], 'contributions:', [contributions[i] for i in pros])
    print(f'cons({candidate_x},{candidate_y}):  ', [courses[i] for i in cons], 'contributions:', [contributions[i] for i in cons])
    
    if not cons:
        print(f'{candidate_x} domine {candidate_y} sur tous les critères.')
        return 'TRIVIAL', []
    
    if not pros:
        print(f'{candidate_y} domine {candidate_x} sur tous les critères.')
        return 'IMPOSSIBLE', []
    
    # Modèle PL
    m = Model('explication_m_1')
    m.params.outputflag = 0
    
    # Variables: x[p,c] = 1 si P est associé à C
    x_vars = {}
    for p in pros:
        for c in cons:
            x_vars[(p,c)] = m.addVar(vtype=GRB.BINARY, name=f'x_{p}_{c}')
    
    # Variables: y[c] = 1 si C est utilisé dans un trade-off (sera forcé à 1)
    y_vars = {}
    for c in cons:
        y_vars[c] = m.addVar(vtype=GRB.BINARY, name=f'y_{c}')
    
    m.update()
    
    epsilon = 0.0
    
    # Contrainte 1: Chaque cons DOIT être couvert (y_c = 1)
    for c in cons:
        m.addConstr(y_vars[c] == 1, name=f'cover_con_{c}')
    
    # Contrainte 2: Chaque pro est utilisé au plus une fois
    for p in pros:
        m.addConstr(quicksum(x_vars[(p,c)] for c in cons) <= 1, name=f'pro_used_once_{p}')
    
    # Contrainte 3: Validité des trade-offs
    # Pour chaque cons, la somme des contributions (cons + ses pros associés) doit être >= 0
    for c in cons:
        sum_contrib = contributions[c] * y_vars[c]
        sum_contrib += quicksum(contributions[p] * x_vars[(p,c)] for p in pros)
        m.addConstr(sum_contrib >= epsilon * y_vars[c], name=f'valid_tradeoff_{c}')
    
    # Contrainte 4: Lien entre x et y (y[c] contrôle quels x peuvent être non-zéro)
    for c in cons:
        for p in pros:
            m.addConstr(x_vars[(p,c)] <= y_vars[c], name=f'link1_{p}_{c}')
        m.addConstr(y_vars[c] <= quicksum(x_vars[(p,c)] for p in pros), name=f'link2_{c}')
    
    # Objectif: minimiser le nombre total de pros utilisés (plutôt que sum(y) qui est constant)
    m.setObjective(quicksum(x_vars.values()), GRB.MINIMIZE)
    m.optimize()
    
    if m.status == GRB.OPTIMAL:
        # Compter le nombre de trade-offs (= nombre de cons couverts = len(cons))
        num_tradeoffs = sum(1 for c in cons if y_vars[c].X > 0.5)
        print(f'\n✓ Explication (m-1) trouvée, longueur = {num_tradeoffs}')
        explanation = []
        for c in cons:
            if y_vars[c].X > 0.5:
                pros_in_tradeoff = [p for p in pros if x_vars[(p,c)].X > 0.5]
                contrib_c = contributions[c]
                contrib_pros = [contributions[p] for p in pros_in_tradeoff]
                total = sum(contrib_pros) + contrib_c
                explanation.append((pros_in_tradeoff, c))
                print(f'  Trade-off: {[courses[p] for p in pros_in_tradeoff]} vs ({courses[c]})')
                print(f'    Contributions: {contrib_pros} + {contrib_c:.1f} = {total:.1f}')
        return 'OPTIMAL', explanation
    else:
        print(f'\n✗ Pas d\'explication (m-1) (certificat de non-existence)')
        return 'INFEASIBLE', []

# Test de l'explication (m-1) pour y > z
result_yz, expl_yz = find_explanation_m_1(y_notes, z_notes, weights, courses, 'y', 'z')


=== Analyse Y > Z (type m-1) ===
pros(Y,Z):   ['Anatomie', 'Chirurgie', 'Forensic', 'Genetique']
cons(Y,Z):   ['Biologie', 'Diagnostic', 'Epidemiologie']
Contributions: [  56  -56    7 -108   -6   20   96]

✓ Explication (m-1) trouvée, longueur = 1
  Trade-off: ['Anatomie', 'Chirurgie', 'Forensic', 'Genetique'] vs (Epidemiologie)
    Contributions: [56, 7, 20, 96] + -6.0 = 173.0
